In [1]:
#http://datareview.info/article/sovremennyie-metodyi-analiza-tonalnosti-teksta/
import pandas
import numpy as np
positive_raw = pandas.read_csv('positive.csv', sep=';', header=None)
negative_raw = pandas.read_csv('negative.csv', sep=';', header=None)
print(type(positive_raw[[3,4]]))
import warnings
warnings.filterwarnings('ignore')

<class 'pandas.core.frame.DataFrame'>


In [2]:
all_raw = positive_raw.append(negative_raw)
all_raw = all_raw[[3, 4]]
all_raw.columns = ['text', 'isPositive']


In [3]:
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def get_normalized_form(word):
    return morph.parse(word)[0].normal_form

regex = re.compile(r"@[\w]+")
def proceed_string(input_str):
    input_str = input_str.replace('\n', ' ')
    input_str = input_str.replace('\\n', ' ')
    input_str = re.sub(r"@[\w]+:", "", input_str)
    input_str = re.sub(r"[^ЁёА-Яа-я\s]", " ", input_str)
    input_str = input_str.lower()
    words_arr = input_str.split()
    words_arr = [get_normalized_form(w) for w in words_arr]
    return words_arr

In [4]:
all_raw['text'] = all_raw['text'].map(proceed_string)

In [5]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(all_raw['text'], all_raw['isPositive'], test_size = 0.2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
n_dim = 200
from gensim.models.word2vec import Word2Vec
twits_w2v = Word2Vec(size=n_dim, min_count=5)
twits_w2v.build_vocab(x_train)
twits_w2v.train(x_train, total_examples=len(x_train), epochs=5)

(7288299, 9988040)

In [13]:
def word_arr_to_vec(arr, length):
    vec = np.zeros(length)
    count = 0
    for word in arr:
        try:
            vec += twits_w2v[word]
            count += 1.0
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec
            

In [14]:
from sklearn.preprocessing import scale
train_vecs = np.array([word_arr_to_vec(arr, 200) for arr in x_train])
train_vecs = scale(train_vecs)


In [15]:
train_vecs

array([[ 0.70178586,  2.4893848 ,  1.58166911, ...,  0.66997997,
        -0.63002943, -1.24321168],
       [ 0.45438809,  0.11922469,  1.36026255, ..., -0.05906654,
         0.88639398, -0.06776256],
       [ 0.97004538,  0.13852308,  0.2736367 , ..., -0.41439339,
         0.12914447, -0.4714326 ],
       ...,
       [ 0.19405161,  0.39043767,  0.48806454, ...,  1.26018358,
         0.86359886, -0.48717387],
       [ 0.84344425,  0.27501822, -0.34013677, ..., -0.13926267,
         0.44107601,  0.09638735],
       [-0.25798638, -0.76816398,  1.04950838, ...,  1.003381  ,
        -1.04009795,  1.25157946]])

In [16]:
#twits_w2v.train(x_test, total_examples=len(x_test), epochs=5)
test_vecs = np.array([word_arr_to_vec(arr, 200) for arr in x_test])
test_vecs = scale(test_vecs)

In [17]:
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vecs, y_train)

print ('Test Accuracy: %.2f'%lr.score(test_vecs, y_test))

Test Accuracy: 0.63


In [21]:
twits_w2v.most_similar(positive = 'плохой')

[('странный', 0.6741325259208679),
 ('страшный', 0.6673437356948853),
 ('хороший', 0.663711428642273),
 ('ужасный', 0.6586064100265503),
 ('важный', 0.6497070789337158),
 ('прекрасный', 0.6259967088699341),
 ('замечательный', 0.6204522848129272),
 ('интересный', 0.6160957217216492),
 ('гавный', 0.613215446472168),
 ('скучный', 0.6131846904754639)]